In [2]:
import torch
PT_FILE_PATH = '/home/wjh/DC/data/VOC2007/VOC2007.pt'
data = torch.load(PT_FILE_PATH)

images_train = data['images_train'] # 假设是 (N, C, H, W) 格式的 Tensor
labels_train = data['labels_train'] # 假设是 (N, NUM_CLASSES) 格式的 Multi-hot Tensor (float)

print(labels_train.shape)

print(labels_train[:10])

torch.Size([5011, 20])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
         0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
     

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# --- 配置参数 ---
# 数据集文件路径 (请根据您的实际路径修改)
DATA_FILES = {
    'train': '/home/wjh/NCFM/dataset/reuters/training_data.csv',
    'test': '/home/wjh/NCFM/dataset/reuters/test_data.csv',
    'validation': '/home/wjh/NCFM/dataset/reuters/val_data.csv'
}
datasets = load_dataset("csv", data_files=DATA_FILES)
print(datasets)

Generating train split: 6769 examples [00:00, 54546.73 examples/s]
Generating test split: 3019 examples [00:00, 69371.59 examples/s]
Generating validation split: 1000 examples [00:00, 50686.45 examples/s]


DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 6769
    })
    test: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 3019
    })
    validation: Dataset({
        features: ['labels', 'sentence'],
        num_rows: 1000
    })
})


In [4]:
print(datasets['train'][0]['labels'])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
import numpy as np

# print([np.ones(32) * c for c in range(90)].size)
np.ones(32).size

32

In [2]:
import torch

syn_data = torch.load('/home/wjh/NCFM/results/condense/condense/reuters/ipc10/sgd_lr_img_0.0100_numr_reqs4096_factor2_20250520-030211/distilled_data/data_1800.pt')

print(len(syn_data))
print(syn_data[0].shape)
print(syn_data[1].shape)

/tmp/ipykernel_85507/3709719734.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  syn_data = torch.load('/home/wjh/NCFM/results/condense/condense/reuters/ipc10/sgd_lr_img_

2
torch.Size([900, 512, 768])
torch.Size([900, 90])
